# Plant Pathology 2021 using RMS Prop with nesterov momentum

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Libraries

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import os
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D,Concatenate 
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.metrics import Precision, Recall

# 1. Dataset

In [ ]:
train_image_path = '/content/gdrive/MyDrive/Datasets/plant-pathology-2021-fgvc8/train_images'
test_image_path = '/content/gdrive/MyDrive/Datasets/plant-pathology-2021-fgvc8/test_images'
train_df_path = '/content/gdrive/MyDrive/Datasets/plant-pathology-2021-fgvc8/train.csv'
test_df_path = '/content/gdrive/MyDrive/Datasets/plant-pathology-2021-fgvc8/sample_submission.csv'

In [ ]:
df_train = pd.read_csv(train_df_path)
df_test=pd.read_csv(test_df_path)

In [ ]:
df_test

In [ ]:
df_train.labels.value_counts()

In [ ]:
plt.figure(figsize=(15,12))
labels = sns.barplot(df_train.labels.value_counts().index,df_train.labels.value_counts())
for item in labels.get_xticklabels():
    item.set_rotation(45)

In [ ]:
print('df_train', df_train.shape)

In [ ]:
print(df_train[0:10])

In [ ]:
img=plt.imread(train_image_path+"/"+df_train["image"][0])

In [ ]:
img.shape

In [ ]:
plt.imshow(img)

In [ ]:
df_train["image"][0]

In [ ]:
img=plt.imread(train_image_path+"/"+df_train["image"][1])

In [ ]:
img.shape

In [ ]:
plt.imshow(img)

In [ ]:
df_train["image"][1]

# 2. Data Augumentation

In [ ]:
HEIGHT = 128
WIDTH= 128
SEED = 42
BATCH_SIZE= 30

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1/255.,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split = 0.2,
    zoom_range = 0.2,
    shear_range = 0.2,
    vertical_flip = False)

In [ ]:
train_dataset = train_datagen.flow_from_dataframe(
    df_train,
    directory = train_image_path,
    x_col = "image",
    y_col = "labels",
    target_size = (HEIGHT,WIDTH),
    class_mode='categorical',
    batch_size = BATCH_SIZE,
    subset = "training",
    shuffle = True,
    seed = SEED,
    validate_filenames = False)

In [ ]:
validation_dataset = train_datagen.flow_from_dataframe(
    df_train,
    directory = train_image_path,
    x_col = "image",
    y_col = "labels",
    target_size = (HEIGHT,WIDTH),
    class_mode='categorical',
    batch_size = BATCH_SIZE,
    subset = "validation",
    shuffle = True,
    seed = SEED,
    validate_filenames = False)

In [ ]:
test_datagen = ImageDataGenerator(
    rescale = 1./255
)

In [ ]:
INPUT_SIZE = (HEIGHT,WIDTH,3)
test_dataset=test_datagen.flow_from_dataframe(
    df_test,
    directory=test_image_path,
    x_col='image',
    y_col=None,
    class_mode=None,
    shuffle = False,
    target_size=INPUT_SIZE[:2])

# 3. Model Training

In [ ]:
df_train["labels"].unique()

array(['healthy', 'scab frog_eye_leaf_spot complex', 'scab', 'complex',
       'rust', 'frog_eye_leaf_spot', 'powdery_mildew',
       'scab frog_eye_leaf_spot', 'frog_eye_leaf_spot complex',
       'rust frog_eye_leaf_spot', 'powdery_mildew complex',
       'rust complex'], dtype=object)

In [ ]:
# inception module

from keras.layers import Conv2D, MaxPooling2D, concatenate
from keras.initializers import lecun_uniform

def inception_module(x, filter_1x1, filter_3x3, filter_5x5):
    # filter_1x1 = 64
    # filter_3x3 = 32
    # filter_5x5 = 16
    conv_1x1_1 = Conv2D(filter_1x1, (1,1), activation='relu', padding='same', kernel_initializer=lecun_uniform(seed=1))(x)
    conv_3x3_1 = Conv2D(filter_3x3, (3,3), activation='relu', padding='same', kernel_initializer=lecun_uniform(seed=1))(conv_1x1_1)
    conv_max_1 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(conv_3x3_1)

    conv_1x1_2 = Conv2D(filter_1x1, (1,1), activation='relu', padding='same', kernel_initializer=lecun_uniform(seed=1))(x)
    conv_5x5_2 = Conv2D(filter_5x5, (5,5), activation='relu', padding='same', kernel_initializer=lecun_uniform(seed=1))(conv_1x1_4)
    conv_2x2_2 = Conv2D(filter_2x2, (2,2), activation='relu', padding='same', kernel_initializer=lecun_uniform(seed=1))(conv_5x5_4)

    output = concatenate([conv_max_1, conv_2x2_2], axis=3)
    return output

In [ ]:
model = Sequential()

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, LeakyReLU

# model = Sequential()

input_shape = Input(shape=(HEIGHT, WIDTH, 3))

x = Conv2D(filters=128, kernel_size=(3, 3), activation=LeakyReLU(), kernel_initializer='he_normal', input_shape=(HEIGHT, WIDTH, 3))(input_shape)
# model.add(MaxPooling2D(pool_size=(2, 2)))
x = Dropout(0.3)(x)
x = Conv2D(filters=64, kernel_size=(3, 3), kernel_initializer='he_uniform', activation="relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

x = inception_module(x, 64, 32, 16)

x = Conv2D(filters=32, kernel_size=(3, 3), activation=LeakyReLU(), kernel_initializer='he_normal')(x)
# model.add(MaxPooling2D(pool_size=(2, 2)))
x = Dropout(0.3)(x)
x = Conv2D(filters=16, kernel_size=(3, 3), kernel_initializer='he_uniform', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

x = inception_module(x, 32, 16, 8)

x = Conv2D(filters=8, kernel_size=(3, 3), kernel_initializer='he_uniform', activation="relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.5)(x)
# model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))
x = Flatten()(x)
x = Dense(4096, activation='relu', kernel_initializer='he_uniform')(x)
x = Dense(1024, activation='relu', kernel_initializer='he_uniform')(x)
x = Dense(128, activation='relu', kernel_initializer='he_uniform')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
output = Dense(6,activation='softmax', kernel_initializer='he_uniform')(x)

In [ ]:
from tensorflow.keras.models import Model
model = Model(inputs=input_shape, outputs=output)

In [ ]:
print(model.summary())

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001, rho = 0.9, momentum = 0.5, nesterov=True),
    loss='categorical_crossentropy',
    metrics=['accuracy', Precision(), Recall()])

In [ ]:
checkpoint_path = "./PP2021_model.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 monitor='val_loss',
                                                 save_best_only=True,
                                                 verbose=1)

In [ ]:
early_stopping = EarlyStopping(
                        monitor='val_loss',
                        min_delta=0.0,
                        patience=5,
                        verbose=1
                )

In [ ]:
fit = model.fit_generator(train_dataset,
                                  validation_data=validation_dataset,
                                  epochs=10,
                                  steps_per_epoch=train_dataset.samples//BATCH_SIZE,
                                  validation_steps=validation_dataset.samples//BATCH_SIZE,
                                  callbacks=[cp_callback, early_stopping]
                                 )

In [ ]:
plt.clf()
fig,ax=plt.subplots(1,2, figsize=(16.0, 6.0))
ax[0].set_xlabel('epochs')
ax[0].plot(np.arange(1, len(fit.history['loss'])+1),
         fit.history['loss'], label='loss')
ax[0].plot(np.arange(1, len(fit.history['loss'])+1),
         fit.history['val_loss'], label='val_loss')
ax[0].set_title("Loss")
ax[0].legend()
ax[1].set_xlabel('epochs')
ax[1].plot(np.arange(1, len(fit.history['accuracy'])+1),
         fit.history['accuracy'], label='accuracy')
ax[1].plot(np.arange(1, len(fit.history['accuracy'])+1),
         fit.history['val_accuracy'], label='val_accuracy')
ax[1].set_title("Accuracy")
ax[1].legend()
plt.show()

# 4. Prediction

In [ ]:
train_dataset.class_indices.items()

In [ ]:
model = tf.keras.models.load_model("./PP2021_model.h5")

In [ ]:
preds = model.predict(test_dataset)
print(preds)

In [ ]:
preds_disease_ind=np.argmax(preds, axis=-1)

In [ ]:
preds_disease_ind

In [ ]:
def get_key(val):
    for key, value in train_dataset.class_indices.items():
        if val == value:
            return key

In [ ]:
for i in range(len(preds_disease_ind)):
    df_test["labels"] [i] = get_key(preds_disease_ind[i])

In [ ]:
df_test

In [ ]:
df_test.to_csv('./submission.csv', index=False)